# Варіаційний метод та секулярне рівняння

## Теоретична основа

### Варіаційний принцип

Варіаційний принцип в квантовій механіці стверджує, що для будь-якої нормованої пробної хвильової функції $\psi$ енергія системи завжди більша або дорівнює точній енергії основного стану:

$$
E_\text{варіаційна} = \frac{\langle \psi | \hat{H} | \psi \rangle}{\langle \psi | \psi \rangle} \geq E_\text{точна}
$$

Це дозволяє нам наближено знаходити енергії та хвильові функції квантових систем.

### Лінійна комбінація атомних орбіталей (LCAO)

У методі LCAO пробна хвильова функція записується як лінійна комбінація базисних функцій:

$$
\psi = \sum_{i=1}^{n} c_i \chi_i
$$

де $\chi_i$ - базисні функції (наприклад, атомні орбіталі), а $c_i$ - варіаційні коефіцієнти.

### Виведення секулярного рівняння

Для системи з двома базисними функціями:
$$
\psi = c_1 \chi_1 + c_2 \chi_2
$$

Варіаційна енергія:
$$
E = \frac{\langle \psi | \hat{H} | \psi \rangle}{\langle \psi | \psi \rangle}
$$

Підставляючи розклад:
$$
E = \frac{c_1^2 H_{11} + 2c_1c_2 H_{12} + c_2^2 H_{22}}{c_1^2 S_{11} + 2c_1c_2 S_{12} + c_2^2 S_{22}}
$$

де:
- $H_{ij} = \langle \chi_i | \hat{H} | \chi_j \rangle$ - матричні елементи гамільтоніана
- $S_{ij} = \langle \chi_i | \chi_j \rangle$ - інтеграли перекриття

### Умова мінімуму енергії

Для знаходження оптимальних коефіцієнтів використовуємо умови:
$$
\frac{\partial E}{\partial c_i} = 0
$$

Це призводить до системи лінійних рівнянь:
$$
\sum_j (H_{ij} - E S_{ij}) c_j = 0
$$

У матричній формі:
$$
(\mathbf{H} - E\mathbf{S})\mathbf{c} = \mathbf{0}
$$

### Секулярне рівняння

Для нетривіального розв'язку детермінант системи повинен дорівнювати нулю:
$$
\det(\mathbf{H} - E\mathbf{S}) = 0
$$

Це **секулярне рівняння**, розв'язки якого дають варіаційні енергії.

### Узагальнена задача на власні значення

Секулярне рівняння можна переписати як узагальнену задачу на власні значення:
$$
\mathbf{H}\mathbf{c} = E\mathbf{S}\mathbf{c}
$$

Або, помноживши зліва на $\mathbf{S}^{-1}$:
$$
\mathbf{S}^{-1}\mathbf{H}\mathbf{c} = E\mathbf{c}
$$

де $E$ - власні значення (енергії), а $\mathbf{c}$ - власні вектори (коефіцієнти розкладу).

In [1]:
from sympy import *
from sympy.physics.hydrogen import Psi_nlm, R_nl, E_nl
from sympy.abc import r, phi,theta, n, l, m, zeta, Z

import numpy as np
import matplotlib.pyplot as plt

## Базисні функції

У нашому прикладі використовуються водневоподібні 1s орбіталі з різними ефективними зарядами ядра $\zeta_1$ та $\zeta_2$:

$$
\chi_1(r) = \sqrt{\frac{\zeta_1^3}{\pi}} e^{-\zeta_1 r}
$$
$$
\chi_2(r) = \sqrt{\frac{\zeta_2^3}{\pi}} e^{-\zeta_2 r}
$$

Ці функції описують 1s орбіталі атома водню з модифікованими ефективними зарядами, що дозволяє врахувати екранування електронів.

In [2]:
zeta_1, zeta_2 = symbols('zeta_1 zeta_2', positive=True)

In [3]:
chi1 = Psi_nlm(1,0,0, r, theta, phi, zeta_1) 
chi2 = Psi_nlm(1,0,0, r, theta, phi, zeta_2) 

## Інтеграли перекриття

Інтеграли перекриття визначають ступінь "перекриття" між базисними функціями:

$$
S_{ij} = \int \chi_i^* \chi_j \, d\tau = \int_0^{\infty} \int_0^{\pi} \int_0^{2\pi} \chi_i^*(r,\theta,\phi) \chi_j(r,\theta,\phi) r^2 \sin\theta \, dr \, d\theta \, d\phi
$$

Для s-орбіталей інтеграція по кутових координатах дає $4\pi$, тому:
$$
S_{ij} = 4\pi \int_0^{\infty} \chi_i(r) \chi_j(r) r^2 \, dr
$$

In [4]:
# Інтеграл перекриття (радіальна частина)
S12 = 4* pi * integrate(chi1 * chi2 * r**2, (r, 0, oo))
S21 = 4* pi * integrate(chi2 * chi1 * r**2, (r, 0, oo))
S11 = 4* pi * integrate(chi1 * chi1 * r**2, (r, 0, oo))
S22 = 4* pi * integrate(chi2 * chi2 * r**2, (r, 0, oo))

## Оператор Гамільтоніана

Для водневоподібного атома гамільтоніан має вигляд:
$$
\hat{H} = -\frac{1}{2}\nabla^2 - \frac{Z}{r}
$$

У сферичних координатах для s-орбіталей (сферично симетричних) лапласіан спрощується до:
$$
\nabla^2 = \frac{1}{r^2}\frac{d}{dr}\left(r^2\frac{d}{dr}\right)
$$

Тому гамільтоніан діє на радіальну частину як:
$$
\hat{H}\chi = -\frac{1}{2}\frac{1}{r^2}\frac{d}{dr}\left(r^2\frac{d\chi}{dr}\right) - \frac{Z}{r}\chi
$$

In [5]:
# оператор гамільтоніана h = -1/2 ∇^2 - Z/r
# для s-орбіталі лапласіан в сферичних координатах:
def h_operator(chi, zeta):
    lap = (1/r**2)*diff(r ** 2 * diff(chi, r), r)
    return -Rational(1,2)*lap - Z/r*chi

## Матричні елементи гамільтоніана

Матричні елементи гамільтоніана обчислюються як:
$$
H_{ij} = \langle \chi_i | \hat{H} | \chi_j \rangle = \int \chi_i^* \hat{H}\chi_j \, d\tau
$$

Ці інтеграли представляють "взаємодію" між базисними функціями через гамільтоніан системи.

- $H_{11}$, $H_{22}$ - діагональні елементи (енергії окремих орбіталей)
- $H_{12} = H_{21}$ - недіагональні елементи (резонансні інтеграли)

In [6]:
# матричні елементи
h_chi1 = h_operator(chi1, zeta_1)
h_chi2 = h_operator(chi2, zeta_2)

In [12]:
H11 = 4*pi * integrate(chi1*h_chi1 * r**2, (r, 0, oo))
H22 = 4*pi * integrate(chi2*h_chi2 * r**2, (r, 0, oo))
H12 = 4*pi * integrate(chi1*h_chi2 * r**2, (r, 0, oo))

H11 = simplify(H11)
H22 = simplify(H22)
H12 = simplify(H12)

## Чисельні значення

Підставляємо конкретні значення параметрів:
- $\zeta_1 = 1.45$ (ефективний заряд для першої орбіталі)
- $\zeta_2 = 2.91$ (ефективний заряд для другої орбіталі)  
- $Z = 2$ (заряд ядра, наприклад, для гелію)

Ці значення можуть відповідати моделюванню атома гелію, де електрони частково екранують один одного.

In [13]:
# Двобазисні інтеграли 
H = Matrix([[H11, H12],
              [H12, H22]])
S = Matrix([[S11, S12],
              [S21, S22]])

S = S.subs([(zeta_1, 1.45), (zeta_2, 2.91), (Z, 2)]) 
H = H.subs([(zeta_1, 1.45), (zeta_2, 2.91), (Z, 2)]) 
H

Matrix([
[         -1.84875, -1.88257712098577],
[-1.88257712098577,          -1.58595]])

In [14]:
# --- Перетворення у NumPy float ---
H = np.array( H.evalf(), dtype=float)
S = np.array(S.evalf(), dtype=float)
H

array([[-1.84875   , -1.88257712],
       [-1.88257712, -1.58595   ]])

# Матриці

Треба розв'язати секулярне рівняння вигляду
$$
(H - ES)\mathbf{c} = 0.
$$

Перетворимо це рівняння до вигляду:
$$
H\mathbf{c} = ES\mathbf{c}.
$$

Помножимо його зліва на $S^{-1}$:
$$
S^{-1}H\mathbf{c} = E\mathbf{c}.
$$

Це рівняння - є рівнянням на власні значення ($E_i$) та власні вектори $\mathbf{c}_i$ матриці $S^{-1}H$.

In [15]:
# Розв'язок generalized eigenvalue problem
eigvals, eigvecs = np.linalg.eig(np.linalg.inv(S) @ H)
eigvals

array([-1.97933487,  1.03047065])

In [16]:
eigvecs

array([[-0.86647437,  0.68999286],
       [-0.49922156, -0.72381618]])

## Інтерпретація результатів

### Власні значення (енергії)
- $E_1 = -1.979$ а.о. - енергія основного стану
- $E_2 = 1.030$ а.о. - енергія збудженого стану

Від'ємне значення першої енергії вказує на зв'язаний стан (електрон утримується ядром).
Додатне значення другої енергії відповідає незв'язаному стану.

### Власні вектори (коефіцієнти MO)
Власні вектори показують, як молекулярні орбіталі утворюються з атомних:

**Основний стан** ($E_1 = -1.979$):
$$
\psi_1 = -0.866\chi_1 - 0.499\chi_2
$$

**Збуджений стан** ($E_2 = 1.030$):
$$
\psi_2 = 0.690\chi_1 - 0.724\chi_2
$$

### Нормування
Перевіряємо нормування першого власного вектора: